In [2]:
import json, os, sys, random, re, math
sys.path.append("../scripts/formalism")
from entropy import *
from formalism_utils import *
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from pprint import pprint
from textblob import Word
import matplotlib.pyplot as plt

In [3]:
import spacy  
nlp = spacy.load("en_core_web_lg") 

In [4]:
J = json.load(open("../data/flickr30k/karpathy/dataset_flickr30k.json", "r"))
annotations = J['images']
print("#captions = ", sum([len(a['sentences']) for a in annotations]))

#captions =  155070


In [8]:
J['images'][0]

{'sentids': [0, 1, 2, 3, 4],
 'imgid': 0,
 'sentences': [{'tokens': ['two',
    'young',
    'guys',
    'with',
    'shaggy',
    'hair',
    'look',
    'at',
    'their',
    'hands',
    'while',
    'hanging',
    'out',
    'in',
    'the',
    'yard'],
   'raw': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.',
   'imgid': 0,
   'sentid': 0},
  {'tokens': ['two',
    'young',
    'white',
    'males',
    'are',
    'outside',
    'near',
    'many',
    'bushes'],
   'raw': 'Two young, White males are outside near many bushes.',
   'imgid': 0,
   'sentid': 1},
  {'tokens': ['two',
    'men',
    'in',
    'green',
    'shirts',
    'are',
    'standing',
    'in',
    'a',
    'yard'],
   'raw': 'Two men in green shirts are standing in a yard.',
   'imgid': 0,
   'sentid': 2},
  {'tokens': ['a',
    'man',
    'in',
    'a',
    'blue',
    'shirt',
    'standing',
    'in',
    'a',
    'garden'],
   'raw': 'A man in a blue shirt standing in

In [5]:
positional_rel_v = ["below", "beneath", "under", "at the bottom of", "above", "on the top of", "on top of", "on"]
positional_rel_h = ["to the left of", "to the right of", "on the left of", "on the right of"]
positional_rel_fb = ["in front of", "behind"]

In [6]:
# See simple statistics, keyword matching
D = defaultdict(list)
for a in tqdm(annotations):
    for s in a['sentences']:
        text = " ".join(s['tokens'])
        for positional_relations in [positional_rel_h, positional_rel_fb, positional_rel_v]:
            for r in positional_relations:
                if f" {r} " in text:
                    tuple = extract_tuple(text, r)
                    #D[r].append(text)
                    if tuple is not None:
                        D[r].append((a['filename'], text, tuple))
                        break
for k in D:
    print(k, len(D[k]))

100%|██████████| 31014/31014 [03:06<00:00, 166.49it/s]

on 35210
on top of 495
in front of 6192
behind 1159
above 486
under 1137
below 50
to the left of 13
on the top of 33
beneath 68
at the bottom of 62
to the right of 7
on the right of 4
on the left of 2


In [19]:
print("#instances = ", sum([len(D[k]) for k in D]))

#instances =  44918


In [8]:
print("#unique images = ", len(set([x[0] for k in D for x in D[k]])))
print("#unique captions = ", len(set([x[1] for k in D for x in D[k]])))
print("#unique concepts = ", len(set([x[2][0] for k in D for x in D[k]] + [x[2][1] for k in D for x in D[k]])))

#unique images =  20784
#unique captions =  43419
#unique concepts =  4205


In [20]:
json.dump(D, open("../data/flickr30k/karpathy/tmp.json", "w"), indent=4)

### Characterize Entropies

In [21]:
entropy_funcs = [
    # "concept_centric_entropy3(num_nouns, num_relations, df)",
    # "concept_centric_entropy4(num_nouns, num_relations, df)",
    # "concept_centric_entropy5(num_nouns, num_relations, df)",
    # "relation_centric_entropy3(num_nouns, num_relations, df)",
    # "relation_centric_entropy4(num_nouns, num_relations, df)",
    # "relation_centric_entropy5(num_nouns, num_relations, df)",
    # "divergence(num_nouns, num_relations, df)",
    # "divergence2(num_nouns, num_relations, df)",
    # "divergence3(num_nouns, num_relations, df)",
    # "concept_role_entropy(num_nouns, df)",
    # "concept_role_entropy2(num_nouns, df)",
    # "role_association(num_nouns, df)",
    # "role_association2(num_nouns, df)",
    # "concept_entropy(num_nouns, df)",
    # "concept_entropy2(num_nouns, df)",
    # "concept_entropy0(num_nouns, df)",
    "concept_role_index_entropy(num_nouns, df)"
]

In [22]:
extracted_tuples = json.load(open("../data/flickr30k/karpathy/tmp.json", "r"))
for r in extracted_tuples:
    print(r, len(extracted_tuples[r]))
print(f"#Total instances with successful SPO parsing = {sum([len(v) for k, v in extracted_tuples.items()])}")


on 35210
on top of 495
in front of 6192
behind 1159
above 486
under 1137
below 50
to the left of 13
on the top of 33
beneath 68
at the bottom of 62
to the right of 7
on the right of 4
on the left of 2
#Total instances with successful SPO parsing = 44918


### Front-Behind

In [23]:
SUBJ, OBJ = [], []
tuples, images, captions = [], [], []
for r in extracted_tuples:
    if not r in positional_rel_fb: continue
    
    for a in extracted_tuples[r]:
        subj, obj, _ = a[-1]
        #singularize
        singlular_subj = Word(subj).lemmatize()
        if not singlular_subj == False: subj = singlular_subj
        singlular_obj = Word(obj).lemmatize() 
        if not singlular_obj == False: obj = singlular_obj

        SUBJ.append(subj)
        OBJ.append(obj)
        tuples.append((subj, obj, r))
        captions.append(a[1])
        images.append(a[0])

print("#unique subjects = ", len(set(SUBJ)))
print("#unique objects = ", len(set(OBJ)))
nouns = sorted(list(set(SUBJ).union(set(OBJ))))
num_nouns = len(nouns)
print("#unique concepts = ", num_nouns)
num_relations = len(positional_rel_fb)
print("#instances = ", len(tuples))
print("#unique instances = ", len(set(tuples)))
print("#unique images = ", len(set(images)))
print("#unique captions = ", len(set(captions)))

n2i = {n:i for i, n in enumerate(nouns)}
r2i = {r:i for i, r in enumerate(positional_rel_fb)}
print(len(r2i), len(n2i))


#unique subjects =  935
#unique objects =  927
#unique concepts =  1483
#instances =  7351
#unique instances =  4368
#unique images =  5333
#unique captions =  7350
2 1483


In [30]:
transpose = True

train_triplets = [] # convert tuple elements to indices

for subj, obj, r in tuples:
    train_triplets.append((n2i[subj], n2i[obj], r2i[r]))
if transpose: train_triplets = Transpose(train_triplets)

df = pd.DataFrame(train_triplets, columns =['O1', 'O2', 'R'])

print("role intrinsic meanings: {} position\n".format("image" if transpose else "linguistic"))
for f in entropy_funcs:
    score = eval(f)
    print(f.split("(")[0], ": ", score, f"({round(score/np.log(2), 2)})") 

role intrinsic meanings: image position

concept_role_index_entropy :  0.3979426141076834 (0.57)


In [31]:
print("role intrinsic meanings: {} position\nA set of complete objects:".format("image" if transpose else "linguistic"))
tmp1, tmp2 = set([t[0] for t in train_triplets]), set([t[1] for t in train_triplets])
cpl1, cpl2 = len(tmp1)/num_nouns, len(tmp2)/num_nouns
complete = tmp1.intersection(tmp2)
print(f"CPL(r1) = {round(cpl1, 2)}, CPL(r2) = {round(cpl2, 2)}, avg = {round((cpl1+cpl2)/2, 2)}")
print(complete)
print(len(complete))

role intrinsic meanings: image position
A set of complete objects:
CPL(r1) = 0.66, CPL(r2) = 0.63, avg = 0.65
{7, 8, 9, 14, 23, 31, 33, 34, 35, 36, 40, 41, 50, 51, 52, 53, 54, 61, 65, 68, 72, 77, 78, 82, 83, 84, 85, 87, 91, 96, 100, 108, 109, 110, 112, 116, 117, 118, 120, 122, 123, 125, 128, 130, 131, 134, 139, 140, 146, 147, 155, 156, 158, 162, 163, 166, 168, 175, 176, 181, 187, 190, 191, 193, 197, 202, 208, 211, 214, 216, 217, 225, 229, 231, 232, 234, 236, 247, 254, 256, 263, 265, 269, 270, 272, 285, 287, 289, 297, 303, 304, 305, 307, 311, 314, 315, 317, 319, 325, 329, 331, 333, 335, 337, 338, 345, 347, 348, 349, 350, 352, 353, 354, 356, 359, 363, 374, 381, 382, 386, 387, 390, 391, 392, 395, 397, 399, 401, 405, 406, 416, 418, 419, 421, 425, 439, 441, 448, 453, 455, 461, 464, 465, 470, 471, 473, 476, 480, 483, 484, 488, 489, 495, 499, 500, 503, 508, 521, 523, 524, 526, 527, 532, 534, 537, 540, 541, 544, 546, 549, 562, 566, 569, 570, 572, 575, 576, 577, 578, 579, 580, 584, 589, 590, 59

In [32]:
# complete nouns under linguistic positions
a = [7, 8, 9, 18, 23, 34, 35, 36, 40, 41, 48, 50, 51, 52, 53, 56, 61, 65, 68, 72, 77, 87, 96, 108, 109, 110, 112, 118, 120, 122, 125, 128, 130, 131, 134, 140, 141, 146, 147, 155, 156, 158, 162, 163, 166, 168, 175, 181, 190, 191, 192, 197, 198, 202, 205, 208, 211, 212, 216, 217, 225, 232, 236, 247, 263, 265, 269, 272, 285, 287, 289, 297, 303, 307, 311, 314, 315, 317, 325, 329, 333, 335, 336, 337, 345, 347, 349, 350, 353, 356, 359, 363, 370, 381, 382, 387, 392, 397, 399, 403, 405, 409, 416, 418, 419, 421, 425, 439, 448, 453, 455, 461, 464, 465, 466, 467, 470, 473, 476, 483, 488, 489, 495, 500, 503, 508, 510, 521, 523, 524, 526, 527, 532, 534, 537, 541, 544, 546, 549, 562, 566, 570, 572, 575, 576, 577, 578, 579, 580, 589, 590, 595, 598, 606, 607, 608, 612, 618, 622, 624, 625, 627, 628, 636, 639, 643, 644, 655, 667, 674, 678, 692, 693, 699, 701, 702, 705, 706, 708, 725, 731, 743, 746, 748, 752, 753, 754, 762, 767, 769, 777, 782, 790, 792, 794, 804, 805, 811, 815, 818, 825, 828, 829, 831, 836, 839, 843, 844, 850, 852, 853, 857, 861, 868, 869, 870, 872, 875, 876, 885, 889, 896, 902, 907, 912, 915, 920, 923, 926, 930, 931, 935, 936, 937, 938, 939, 940, 941, 943, 945, 955, 957, 958, 962, 965, 966, 970, 973, 974, 977, 978, 979, 981, 983, 987, 988, 990, 995, 1001, 1014, 1017, 1027, 1028, 1030, 1033, 1034, 1037, 1052, 1057, 1058, 1069, 1071, 1081, 1082, 1087, 1089, 1092, 1098, 1104, 1108, 1111, 1113, 1114, 1117, 1126, 1135, 1136, 1140, 1147, 1154, 1156, 1157, 1166, 1170, 1171, 1175, 1181, 1185, 1192, 1196, 1198, 1203, 1206, 1209, 1211, 1219, 1220, 1232, 1233, 1235, 1236, 1237, 1242, 1246, 1247, 1251, 1252, 1255, 1256, 1257, 1258, 1260, 1265, 1266, 1271, 1272, 1273, 1279, 1280, 1282, 1283, 1284, 1287, 1294, 1298, 1302, 1304, 1311, 1322, 1329, 1336, 1351, 1352, 1353, 1355, 1359, 1360, 1363, 1366, 1367, 1370, 1376, 1380, 1387, 1391, 1400, 1402, 1404, 1406, 1409, 1418, 1425, 1432, 1435, 1441, 1446, 1447, 1449, 1453, 1459, 1465, 1466, 1468, 1469, 1471, 1473, 1475, 1479, 1481]
print(len(a))
# complete nouns under image positions
b = [7, 8, 9, 14, 23, 31, 33, 34, 35, 36, 40, 41, 50, 51, 52, 53, 54, 61, 65, 68, 72, 77, 78, 82, 83, 84, 85, 87, 91, 96, 100, 108, 109, 110, 112, 116, 117, 118, 120, 122, 123, 125, 128, 130, 131, 134, 139, 140, 146, 147, 155, 156, 158, 162, 163, 166, 168, 175, 176, 181, 187, 190, 191, 193, 197, 202, 208, 211, 214, 216, 217, 225, 229, 231, 232, 234, 236, 247, 254, 256, 263, 265, 269, 270, 272, 285, 287, 289, 297, 303, 304, 305, 307, 311, 314, 315, 317, 319, 325, 329, 331, 333, 335, 337, 338, 345, 347, 348, 349, 350, 352, 353, 354, 356, 359, 363, 374, 381, 382, 386, 387, 390, 391, 392, 395, 397, 399, 401, 405, 406, 416, 418, 419, 421, 425, 439, 441, 448, 453, 455, 461, 464, 465, 470, 471, 473, 476, 480, 483, 484, 488, 489, 495, 499, 500, 503, 508, 521, 523, 524, 526, 527, 532, 534, 537, 540, 541, 544, 546, 549, 562, 566, 569, 570, 572, 575, 576, 577, 578, 579, 580, 584, 589, 590, 595, 604, 607, 608, 612, 618, 621, 622, 624, 627, 628, 635, 636, 639, 643, 644, 649, 655, 656, 664, 672, 674, 678, 692, 693, 695, 699, 702, 705, 706, 710, 715, 717, 725, 726, 731, 743, 746, 748, 752, 753, 754, 757, 762, 767, 785, 790, 792, 794, 797, 804, 805, 815, 818, 825, 828, 829, 831, 836, 843, 844, 850, 852, 857, 860, 861, 868, 869, 870, 876, 877, 879, 880, 885, 891, 894, 896, 902, 907, 911, 912, 915, 920, 926, 930, 931, 935, 936, 937, 938, 939, 940, 941, 943, 945, 954, 955, 957, 958, 962, 966, 969, 970, 973, 974, 977, 978, 979, 983, 985, 987, 988, 990, 995, 1001, 1014, 1027, 1030, 1033, 1034, 1037, 1038, 1049, 1052, 1054, 1057, 1069, 1071, 1073, 1081, 1089, 1090, 1092, 1098, 1101, 1104, 1108, 1111, 1113, 1114, 1117, 1123, 1126, 1128, 1135, 1136, 1147, 1149, 1151, 1153, 1154, 1156, 1157, 1162, 1166, 1170, 1171, 1175, 1177, 1178, 1181, 1187, 1189, 1192, 1196, 1198, 1203, 1206, 1209, 1211, 1219, 1220, 1221, 1227, 1232, 1235, 1236, 1237, 1239, 1242, 1243, 1246, 1247, 1251, 1252, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1265, 1266, 1271, 1272, 1273, 1281, 1287, 1290, 1298, 1302, 1304, 1305, 1309, 1311, 1316, 1321, 1322, 1329, 1336, 1350, 1351, 1352, 1353, 1355, 1359, 1360, 1362, 1363, 1366, 1367, 1369, 1370, 1376, 1380, 1385, 1387, 1390, 1391, 1395, 1400, 1402, 1404, 1406, 1413, 1418, 1419, 1425, 1435, 1441, 1446, 1447, 1449, 1450, 1453, 1459, 1464, 1465, 1466, 1468, 1469, 1471, 1473, 1474, 1475, 1479, 1481]
print(len(b))
print("A set of nouns with complete support under both role meanings:")
both_complete_nouns = [n for i, n in enumerate(nouns) if i in a and i in b]
print(both_complete_nouns)
print(len(both_complete_nouns))
print("#instances formed by both_complete_nouns: ", 
      len([t for t in tuples if t[0] in both_complete_nouns and t[1] in both_complete_nouns]))

379
442
A set of nouns with complete support under both role meanings:
['adolescent', 'adult', 'advertisement', 'animal', 'area', 'arena', 'arm', 'art', 'artist', 'audience', 'automobile', 'baby', 'back', 'bag', 'ball', 'balloon', 'band', 'banner', 'basket', 'beach', 'bench', 'beverage', 'bicycle', 'bike', 'bird', 'black', 'blanket', 'block', 'board', 'boat', 'body', 'book', 'bottle', 'box', 'boy', 'brick', 'bride', 'bridge', 'bubble', 'bucket', 'building', 'bull', 'bus', 'butcher', 'cage', 'cake', 'camera', 'can', 'canoe', 'canvas', 'car', 'card', 'cart', 'cat', 'cattle', 'chair', 'child', 'choir', 'cigarette', 'city', 'clothes', 'clothing', 'clown', 'coin', 'competition', 'concert', 'cone', 'construction', 'container', 'contraption', 'corner', 'couch', 'couple', 'court', 'cow', 'cross', 'crosswalk', 'crowd', 'crown', 'curb', 'customer', 'cyclist', 'dancer', 'desk', 'device', 'dinosaur', 'dish', 'dock', 'dog', 'door', 'drink', 'drum', 'drummer', 'duck', 'dust', 'elephant', 'equipment'

In [33]:
heatmap = {}
for transpose in [False, True]:
    train_triplets = []
    for subj, obj, r in tuples:
        train_triplets.append((n2i[subj], n2i[obj], r2i[r]))
    if transpose: train_triplets = Transpose(train_triplets)

    c = Counter(train_triplets)
    print(c.most_common(3))

    prefix = "image" if transpose else "linguistic"
    heatmap[prefix] = np.zeros((len(both_complete_nouns), len(both_complete_nouns)))
    X = []
    for k, o1 in enumerate(both_complete_nouns):
        for l, o2 in enumerate(both_complete_nouns):
            x = c[(n2i[o1], n2i[o2], 0)] + c[(n2i[o1], n2i[o2], 1)]
            X.append(x)
            heatmap[prefix][k][l] = x


[((767, 166, 0), 129), ((926, 166, 0), 117), ((767, 1425, 0), 71)]
[((767, 166, 0), 129), ((926, 166, 0), 117), ((767, 1425, 0), 71)]


In [34]:
dead = []
iters = 0
killed_this_iter = True
while np.sum(heatmap['image']) * np.sum(heatmap['linguistic']) > 0 and killed_this_iter:
    killed_this_iter = False
    sum_axis0, sum_axis1 = np.sum(heatmap['image'], axis=0), np.sum(heatmap['image'], axis=1)
    for i, s in enumerate(sum_axis0):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop row i
                heatmap['image'][i][j] = 0
                # drop row i and col i
                heatmap['linguistic'][i][j] = 0
                heatmap['linguistic'][j][i] = 0

            dead.append(i)
            killed_this_iter = True
    for i, s in enumerate(sum_axis1):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop col i
                heatmap['image'][j][i] = 0
                # drop row i and col i
                heatmap['linguistic'][i][j] = 0
                heatmap['linguistic'][j][i] = 0
            dead.append(i)
            killed_this_iter = True

    sum_axis0, sum_axis1 = np.sum(heatmap['linguistic'], axis=0), np.sum(heatmap['linguistic'], axis=1)
    for i, s in enumerate(sum_axis0):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop row i
                heatmap['linguistic'][i][j] = 0
                # drop row i and col i
                heatmap['image'][i][j] = 0
                heatmap['image'][j][i] = 0

            dead.append(i)
            killed_this_iter = True
    for i, s in enumerate(sum_axis1):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop col i
                heatmap['linguistic'][j][i] = 0
                # drop row i and col i
                heatmap['image'][i][j] = 0
                heatmap['image'][j][i] = 0
            dead.append(i)
            killed_this_iter = True

    iters += 1
    print(f"dead nouns = {len(dead)}")
    print(f"""finish iter {iters}, 
          remaining examples = {np.sum(heatmap['image'])} (image), 
          {np.sum(heatmap['linguistic'])} (linguistic)""")    


dead nouns = 59
finish iter 1, 
          remaining examples = 4061.0 (image), 
          4061.0 (linguistic)
dead nouns = 64
finish iter 2, 
          remaining examples = 4018.0 (image), 
          4018.0 (linguistic)
dead nouns = 65
finish iter 3, 
          remaining examples = 4016.0 (image), 
          4016.0 (linguistic)
dead nouns = 65
finish iter 4, 
          remaining examples = 4016.0 (image), 
          4016.0 (linguistic)


In [35]:
alive = set(list(range(len(both_complete_nouns)))) - set(dead)
alive = sorted(list(alive))
print("nouns with complete support".format(alive))
print(len(alive))
rows = np.array(alive, dtype=np.intp)
columns = np.array(alive, dtype=np.intp)

bijective_heatmap = heatmap['linguistic'][np.ix_(rows, columns)]
print("sanity check (linguistic): every row sum or col sum is positive")
print(np.min(np.sum(bijective_heatmap, axis=0)), np.min(np.sum(bijective_heatmap, axis=1)))

bijective_heatmap = heatmap['image'][np.ix_(rows, columns)]
print("sanity check (image): every row sum or col sum is positive")
print(np.min(np.sum(bijective_heatmap, axis=0)), np.min(np.sum(bijective_heatmap, axis=1)))

crux = alive # They are both_complete_nouns' indices
print(len(crux))
crux_nouns = [both_complete_nouns[i] for i in crux]


nouns with complete support
265
sanity check (linguistic): every row sum or col sum is positive
1.0 1.0
sanity check (image): every row sum or col sum is positive
1.0 1.0
265


In [36]:
for transpose in [False, True]:
    print("role intrinsic meanings: {} position\n".format("image" if transpose else "linguistic"))
    SUBJ, OBJ = [], []
    tuples, images, captions = [], [], []
    for r in extracted_tuples:
        if not r in positional_rel_fb: continue
        
        for a in extracted_tuples[r]:
            subj, obj, _ = a[-1]
            #singularize
            singlular_subj = Word(subj).lemmatize()
            if not singlular_subj == False: subj = singlular_subj
            singlular_obj = Word(obj).lemmatize() 
            if not singlular_obj == False: obj = singlular_obj

            if subj in crux_nouns and obj in crux_nouns:
                SUBJ.append(subj)
                OBJ.append(obj)
                tuples.append((subj, obj, r))
                captions.append(a[1])
                images.append(a[0])
    
    nouns = sorted(list(set(SUBJ).union(set(OBJ))))
    num_nouns = len(nouns)
    print("#unique concepts = ", num_nouns)
    num_relations = len(positional_rel_fb)
    print("#instances = ", len(tuples))
    print("#unique instances = ", len(set(tuples)))
    print("#unique images = ", len(set(images)))
    print("#unique captions = ", len(set(captions)))
    
    train_triplets = [] # convert tuple elements to indices
    n2i = {n:i for i, n in enumerate(nouns)}
    r2i = {r:i for i, r in enumerate(positional_rel_fb)}
    print(len(r2i), len(n2i), "\n")
    for subj, obj, r in tuples:
        train_triplets.append((n2i[subj], n2i[obj], r2i[r]))
    if transpose: train_triplets = Transpose(train_triplets)

    # Sanity check crux_nouns are indeed complete under both linguistic & image positional roles
    print("#unique O1 = ", len(set([t[0] for t in train_triplets])))
    print("#unique O2 = ", len(set([t[1] for t in train_triplets])))
    df = pd.DataFrame(train_triplets, columns =['O1', 'O2', 'R'])
    for f in entropy_funcs:
        score = eval(f)
        print(f.split("(")[0], ": ", score, f"({round(score/np.log(2), 2)})")  
    print("--------------------------------------------------------\n\n")

role intrinsic meanings: linguistic position

#unique concepts =  265
#instances =  4016
#unique instances =  1773
#unique images =  3185
#unique captions =  4016
2 265 

#unique O1 =  265
#unique O2 =  265
concept_role_index_entropy :  0.3294425814619563 (0.48)
--------------------------------------------------------


role intrinsic meanings: image position

#unique concepts =  265
#instances =  4016
#unique instances =  1773
#unique images =  3185
#unique captions =  4016
2 265 

#unique O1 =  265
#unique O2 =  265
concept_role_index_entropy :  0.45434696276899766 (0.66)
--------------------------------------------------------




In [37]:
if transpose:
    print(len(train_triplets))
    print("train_triplets under {} positional roles:".format("image" if transpose else "linguistic"))
    print(train_triplets)
    print("unique train_triplets = {}".format(len(set(train_triplets))))

4016
train_triplets under image positional roles:
[(259, 36, 0), (259, 36, 0), (97, 42, 0), (109, 151, 0), (132, 145, 0), (72, 248, 0), (134, 34, 0), (156, 257, 0), (132, 238, 0), (28, 253, 0), (28, 253, 0), (107, 102, 0), (132, 147, 0), (132, 45, 0), (134, 34, 0), (30, 39, 0), (207, 207, 0), (12, 7, 0), (107, 34, 0), (134, 202, 0), (132, 34, 0), (204, 40, 0), (132, 34, 0), (132, 204, 0), (132, 204, 0), (259, 34, 0), (158, 73, 0), (156, 34, 0), (262, 253, 0), (156, 254, 0), (218, 73, 0), (110, 241, 0), (259, 223, 0), (40, 80, 0), (142, 156, 0), (132, 64, 0), (176, 199, 0), (156, 3, 0), (97, 73, 0), (132, 217, 0), (47, 257, 0), (28, 223, 0), (132, 73, 0), (97, 132, 0), (97, 180, 0), (132, 43, 0), (28, 43, 0), (156, 34, 0), (131, 62, 0), (28, 132, 0), (134, 62, 0), (97, 253, 0), (132, 62, 0), (158, 31, 0), (131, 241, 0), (134, 241, 0), (25, 124, 0), (134, 124, 0), (156, 34, 0), (191, 34, 0), (132, 75, 0), (184, 53, 0), (132, 84, 0), (134, 84, 0), (156, 34, 0), (262, 34, 0), (132, 223, 0)

### Top-Bottom (merged)

In [39]:
# positional_rel_v = ["above", "below", "beneath", "on", "on top of", "under"]
map = {
    "above": "top",
    "on": "top", 
    "on top of": "top",
    "on the top of": "top",  
    "at the bottom of": "bottom",
    "below": "bottom", 
    "beneath": "bottom",
    "under": "bottom"
}
relations = ["top", "bottom"]

SUBJ, OBJ = [], []
tuples, images, captions = [], [], []
for unmapped_r in extracted_tuples:
    if not unmapped_r in positional_rel_v: continue
    
    r = map[unmapped_r]

    for a in extracted_tuples[unmapped_r]:
        subj, obj, _ = a[-1]
        #singularize
        singlular_subj = Word(subj).lemmatize()
        if not singlular_subj == False: subj = singlular_subj
        singlular_obj = Word(obj).lemmatize() 
        if not singlular_obj == False: obj = singlular_obj

        SUBJ.append(subj)
        OBJ.append(obj)
        tuples.append((subj, obj, r))
        captions.append(a[1])
        images.append(a[0])

print("#unique subjects = ", len(set(SUBJ)))
print("#unique objects = ", len(set(OBJ)))
nouns = sorted(list(set(SUBJ).union(set(OBJ))))
num_nouns = len(nouns)
print("#unique concepts = ", num_nouns)
num_relations = len(relations)
print("#instances = ", len(tuples))
print("#unique instances = ", len(set(tuples)))
print("#unique images = ", len(set(images)))
print("#unique captions = ", len(set(captions)))

n2i = {n:i for i, n in enumerate(nouns)}
r2i = {r:i for i, r in enumerate(relations)}
print(len(r2i), len(n2i))


#unique subjects =  2269
#unique objects =  1717
#unique concepts =  3003
#instances =  37541
#unique instances =  13766
#unique images =  18436
#unique captions =  37462
2 3003


In [42]:
transpose = True

train_triplets = [] # convert tuple elements to indices

for subj, obj, r in tuples:
    train_triplets.append((n2i[subj], n2i[obj], r2i[r]))
    
if transpose: train_triplets = Transpose(train_triplets)

df = pd.DataFrame(train_triplets, columns =['O1', 'O2', 'R'])

print("role intrinsic meanings: {} position\n".format("image" if transpose else "linguistic"))
for f in entropy_funcs:
    score = eval(f)
    print(f.split("(")[0], ": ", score, f"({round(score/np.log(2), 2)})") 

role intrinsic meanings: image position

concept_role_index_entropy :  0.2470569172321184 (0.36)


In [43]:
print("role intrinsic meanings: {} position\nA set of complete objects:".format("image" if transpose else "linguistic"))
tmp1, tmp2 = set([t[0] for t in train_triplets]), set([t[1] for t in train_triplets])
cpl1, cpl2 = len(tmp1)/num_nouns, len(tmp2)/num_nouns
complete = tmp1.intersection(tmp2)
print(f"CPL(r1) = {round(cpl1, 2)}, CPL(r2) = {round(cpl2, 2)}, avg = {round((cpl1+cpl2)/2, 2)}")
print(complete)
print(len(complete))

role intrinsic meanings: image position
A set of complete objects:
CPL(r1) = 0.76, CPL(r2) = 0.58, avg = 0.67
{2050, 2051, 2052, 5, 6, 2057, 2059, 12, 13, 14, 2063, 2064, 17, 2068, 2069, 2070, 24, 2072, 2073, 29, 30, 2079, 33, 2081, 41, 2091, 48, 2101, 57, 58, 67, 2115, 2116, 2122, 75, 76, 2123, 2127, 2130, 2131, 2132, 85, 86, 2134, 88, 2137, 2138, 91, 2139, 2141, 94, 96, 99, 2148, 101, 103, 104, 2151, 2153, 107, 2154, 2156, 111, 112, 113, 2159, 2162, 2163, 117, 2165, 2167, 120, 2169, 123, 124, 125, 2171, 2174, 128, 2177, 131, 138, 2187, 2188, 141, 2189, 2190, 2192, 145, 146, 147, 148, 2197, 150, 2200, 153, 2203, 2204, 157, 158, 2205, 2206, 2208, 2210, 163, 164, 168, 2216, 170, 171, 2218, 2221, 2223, 179, 180, 181, 2228, 183, 2233, 187, 188, 2236, 190, 2240, 2244, 198, 201, 2250, 203, 2253, 2254, 207, 2256, 209, 212, 213, 2260, 217, 2266, 2269, 222, 2271, 228, 229, 230, 231, 233, 2281, 2282, 237, 238, 239, 240, 2285, 242, 2287, 2289, 2294, 248, 249, 2297, 251, 2298, 2300, 2301, 2302, 2

In [44]:
# complete nouns under linguistic positions
a = [2050, 2051, 2052, 5, 6, 2053, 2057, 2059, 12, 14, 2063, 2064, 17, 2068, 2069, 2070, 24, 2072, 2073, 2075, 29, 30, 2079, 33, 41, 2091, 48, 2101, 57, 58, 67, 2115, 72, 75, 76, 2123, 2127, 2130, 2131, 84, 85, 86, 2132, 88, 2133, 2134, 91, 2137, 2138, 2139, 2141, 96, 2148, 101, 103, 104, 2151, 2154, 107, 2156, 111, 112, 113, 2159, 2162, 116, 117, 2163, 2165, 120, 2167, 2169, 123, 124, 125, 2171, 2174, 128, 2177, 131, 138, 2187, 2188, 2189, 2190, 2192, 145, 146, 147, 148, 2197, 2200, 153, 2203, 2204, 157, 158, 2205, 2206, 2208, 2210, 163, 164, 168, 2216, 170, 171, 2218, 2221, 2223, 179, 180, 181, 2228, 183, 2233, 186, 187, 188, 2236, 190, 2240, 2244, 198, 201, 2253, 2254, 207, 2256, 209, 212, 213, 2260, 217, 2266, 2269, 222, 2271, 227, 228, 229, 230, 231, 233, 2281, 2282, 237, 238, 239, 240, 2285, 242, 2287, 2289, 2290, 2294, 249, 2297, 251, 2298, 2300, 2301, 260, 261, 2308, 2309, 264, 2310, 266, 2313, 2316, 270, 2318, 272, 2321, 2322, 2325, 2326, 279, 280, 2327, 2328, 283, 284, 285, 286, 2331, 289, 2337, 291, 2338, 293, 294, 2341, 298, 299, 302, 303, 2350, 2351, 306, 2355, 2356, 2357, 310, 311, 2361, 314, 315, 316, 2362, 2365, 320, 2369, 324, 325, 2372, 2373, 2376, 2379, 332, 334, 336, 2386, 2388, 344, 2393, 2395, 2396, 349, 351, 2401, 2403, 356, 359, 2407, 361, 2410, 363, 2413, 370, 371, 2421, 2422, 2424, 378, 380, 2428, 2430, 2434, 2436, 2438, 391, 393, 2442, 395, 396, 2443, 398, 399, 2453, 406, 407, 2454, 2456, 2461, 2462, 415, 416, 417, 418, 419, 2463, 421, 422, 2466, 2467, 426, 427, 428, 429, 2474, 431, 2475, 2480, 2481, 2482, 2483, 437, 2485, 440, 441, 442, 443, 2489, 445, 2490, 2492, 2493, 449, 450, 2497, 452, 2498, 2499, 455, 456, 2501, 458, 2504, 2505, 2506, 462, 2507, 2509, 465, 2512, 467, 468, 2513, 470, 2515, 2516, 2517, 474, 475, 2519, 477, 2520, 2525, 480, 2526, 2531, 484, 2535, 2536, 489, 490, 2537, 492, 493, 2539, 2541, 496, 2542, 498, 2547, 502, 2551, 505, 2553, 2555, 509, 513, 2561, 515, 2562, 2563, 2567, 2568, 2569, 2570, 2571, 526, 2576, 529, 2577, 532, 2580, 535, 2583, 2584, 2585, 2586, 541, 2589, 543, 2590, 545, 2594, 2595, 553, 2602, 555, 556, 557, 2603, 2604, 2606, 561, 2607, 563, 566, 567, 569, 570, 2617, 2619, 573, 574, 575, 576, 2623, 2624, 579, 2628, 583, 584, 2631, 2633, 587, 2636, 591, 2642, 2643, 597, 2646, 2650, 2652, 2655, 2657, 2658, 2659, 614, 2662, 617, 2666, 2667, 620, 2670, 624, 625, 626, 2673, 2674, 2675, 630, 631, 632, 2676, 2680, 2683, 639, 640, 643, 2694, 650, 2698, 654, 2702, 657, 658, 2705, 2707, 2708, 2712, 665, 2716, 669, 2718, 671, 672, 673, 2720, 675, 676, 677, 2722, 2724, 2726, 682, 684, 686, 687, 688, 2742, 2745, 2746, 2747, 2748, 701, 2749, 2751, 2754, 707, 708, 2755, 710, 2756, 712, 2757, 2761, 2765, 2767, 720, 2770, 726, 2775, 728, 731, 732, 2780, 735, 2784, 2786, 740, 741, 2790, 2791, 2792, 745, 2793, 2794, 2796, 749, 751, 752, 2802, 758, 759, 2806, 2809, 762, 2811, 764, 765, 2814, 2815, 2819, 775, 2824, 777, 778, 2825, 2827, 784, 2835, 2839, 792, 793, 2841, 797, 2846, 799, 803, 805, 2854, 2857, 2860, 817, 2866, 2867, 820, 822, 823, 2873, 2874, 827, 2876, 830, 2879, 832, 833, 2882, 837, 2887, 2888, 844, 2893, 847, 848, 849, 2898, 851, 2899, 2901, 2902, 2905, 858, 2911, 2912, 870, 875, 2923, 2925, 878, 2929, 2931, 2932, 890, 891, 895, 897, 898, 2947, 2950, 2955, 908, 2957, 910, 2959, 912, 2962, 2963, 917, 2965, 2968, 921, 2969, 2970, 924, 2972, 929, 930, 2977, 933, 935, 2983, 2986, 943, 948, 950, 2998, 956, 957, 964, 975, 976, 977, 981, 982, 983, 985, 988, 990, 993, 999, 1001, 1004, 1007, 1014, 1020, 1021, 1022, 1025, 1027, 1030, 1041, 1044, 1046, 1047, 1058, 1059, 1061, 1062, 1063, 1065, 1066, 1068, 1074, 1075, 1078, 1080, 1090, 1093, 1094, 1096, 1099, 1100, 1105, 1106, 1111, 1115, 1117, 1120, 1121, 1122, 1127, 1133, 1134, 1138, 1144, 1150, 1153, 1159, 1163, 1166, 1171, 1173, 1176, 1178, 1179, 1182, 1183, 1187, 1190, 1191, 1195, 1197, 1200, 1204, 1208, 1211, 1221, 1224, 1227, 1230, 1232, 1235, 1238, 1245, 1246, 1247, 1250, 1260, 1263, 1264, 1271, 1276, 1278, 1280, 1282, 1284, 1287, 1288, 1289, 1290, 1291, 1292, 1294, 1299, 1309, 1315, 1321, 1325, 1326, 1327, 1337, 1343, 1346, 1351, 1355, 1356, 1357, 1364, 1365, 1367, 1369, 1375, 1377, 1389, 1390, 1398, 1400, 1409, 1411, 1414, 1416, 1417, 1424, 1426, 1434, 1435, 1436, 1440, 1443, 1444, 1445, 1447, 1448, 1449, 1451, 1459, 1462, 1465, 1467, 1468, 1471, 1474, 1476, 1477, 1481, 1483, 1486, 1490, 1491, 1498, 1504, 1505, 1519, 1521, 1522, 1525, 1530, 1531, 1532, 1533, 1538, 1539, 1541, 1542, 1543, 1544, 1549, 1551, 1553, 1555, 1556, 1562, 1565, 1569, 1570, 1580, 1582, 1583, 1584, 1586, 1589, 1592, 1594, 1595, 1604, 1605, 1606, 1615, 1616, 1619, 1620, 1623, 1628, 1639, 1642, 1644, 1647, 1649, 1652, 1653, 1657, 1658, 1659, 1660, 1663, 1665, 1669, 1671, 1676, 1679, 1681, 1683, 1685, 1690, 1695, 1700, 1701, 1707, 1711, 1712, 1716, 1717, 1718, 1720, 1721, 1723, 1724, 1726, 1732, 1735, 1737, 1740, 1742, 1745, 1747, 1749, 1758, 1764, 1769, 1770, 1771, 1775, 1777, 1782, 1783, 1784, 1785, 1787, 1791, 1794, 1795, 1796, 1801, 1804, 1806, 1809, 1810, 1818, 1820, 1822, 1823, 1825, 1826, 1827, 1828, 1836, 1837, 1838, 1839, 1840, 1841, 1844, 1846, 1847, 1850, 1863, 1864, 1865, 1869, 1871, 1875, 1878, 1881, 1887, 1888, 1889, 1892, 1894, 1896, 1898, 1899, 1904, 1907, 1908, 1911, 1913, 1916, 1918, 1919, 1920, 1921, 1922, 1923, 1926, 1928, 1933, 1935, 1936, 1938, 1939, 1940, 1941, 1943, 1949, 1953, 1955, 1957, 1958, 1969, 1970, 1971, 1973, 1975, 1976, 1977, 1980, 1983, 1985, 1987, 1989, 1993, 1997, 2000, 2006, 2012, 2014, 2015, 2020, 2021, 2022, 2031, 2038, 2042, 2043, 2044, 2045]
# complete nouns under image positions
b = [2050, 2051, 2052, 5, 6, 2057, 2059, 12, 13, 14, 2063, 2064, 17, 2068, 2069, 2070, 24, 2072, 2073, 29, 30, 2079, 33, 2081, 41, 2091, 48, 2101, 57, 58, 67, 2115, 2116, 2122, 75, 76, 2123, 2127, 2130, 2131, 2132, 85, 86, 2134, 88, 2137, 2138, 91, 2139, 2141, 94, 96, 99, 2148, 101, 103, 104, 2151, 2153, 107, 2154, 2156, 111, 112, 113, 2159, 2162, 2163, 117, 2165, 2167, 120, 2169, 123, 124, 125, 2171, 2174, 128, 2177, 131, 138, 2187, 2188, 141, 2189, 2190, 2192, 145, 146, 147, 148, 2197, 150, 2200, 153, 2203, 2204, 157, 158, 2205, 2206, 2208, 2210, 163, 164, 168, 2216, 170, 171, 2218, 2221, 2223, 179, 180, 181, 2228, 183, 2233, 187, 188, 2236, 190, 2240, 2244, 198, 201, 2250, 203, 2253, 2254, 207, 2256, 209, 212, 213, 2260, 217, 2266, 2269, 222, 2271, 228, 229, 230, 231, 233, 2281, 2282, 237, 238, 239, 240, 2285, 242, 2287, 2289, 2294, 248, 249, 2297, 251, 2298, 2300, 2301, 2302, 256, 2303, 260, 261, 2308, 2309, 264, 2310, 266, 2313, 268, 2316, 2318, 272, 2321, 2322, 2325, 2326, 279, 280, 2327, 283, 284, 285, 286, 2331, 2334, 289, 2337, 291, 2338, 293, 294, 2341, 298, 299, 302, 303, 304, 2350, 306, 2351, 2355, 2356, 310, 311, 2357, 2361, 314, 315, 316, 2362, 2363, 2365, 320, 2369, 2370, 324, 325, 2372, 2373, 2376, 2379, 332, 334, 2383, 336, 2386, 2388, 344, 2393, 2395, 2396, 349, 351, 2401, 2403, 356, 359, 2407, 361, 2410, 363, 2413, 370, 371, 2421, 2422, 2424, 378, 380, 2428, 2430, 2434, 2436, 2437, 2438, 391, 393, 2442, 395, 396, 2443, 398, 399, 2453, 406, 407, 2454, 2456, 2461, 2462, 415, 416, 417, 418, 419, 2464, 421, 422, 2466, 2467, 426, 427, 428, 429, 2474, 431, 432, 2475, 2480, 2481, 2482, 437, 2485, 440, 441, 442, 443, 2489, 445, 2492, 2493, 449, 450, 2497, 452, 453, 2498, 455, 456, 2499, 458, 2501, 2504, 2505, 462, 2506, 2507, 465, 2509, 467, 468, 2512, 470, 2513, 2515, 2516, 474, 475, 2517, 477, 2519, 2520, 480, 2525, 2526, 2531, 2535, 2536, 489, 490, 2537, 492, 493, 2539, 2541, 496, 2542, 498, 2547, 502, 2551, 505, 2553, 507, 2555, 509, 513, 2561, 515, 2562, 2563, 2567, 2568, 2569, 2570, 2571, 526, 2576, 529, 2577, 532, 2580, 535, 2583, 2584, 2585, 2586, 2587, 541, 2589, 543, 2590, 545, 553, 2602, 555, 556, 557, 2603, 2604, 2606, 561, 2607, 563, 566, 567, 568, 569, 570, 2617, 572, 573, 574, 575, 576, 2623, 2624, 579, 2628, 583, 584, 2633, 587, 2636, 591, 2642, 2643, 597, 2646, 599, 2650, 603, 2652, 2655, 2657, 2658, 2659, 614, 2662, 617, 2666, 2667, 620, 2670, 624, 625, 2673, 2674, 2675, 2676, 630, 631, 632, 2680, 2683, 639, 640, 643, 2694, 650, 2698, 654, 2702, 657, 658, 2705, 2707, 2708, 2712, 665, 2716, 669, 2718, 671, 672, 673, 2720, 675, 2722, 677, 2724, 2726, 2729, 682, 684, 686, 687, 688, 2742, 2743, 2745, 2746, 2747, 700, 701, 2748, 2749, 2751, 2753, 2754, 707, 708, 2755, 710, 711, 712, 2756, 2757, 2761, 2765, 2767, 720, 2770, 726, 2775, 728, 731, 732, 2779, 2780, 735, 2784, 2786, 740, 741, 2790, 2791, 2792, 745, 2793, 2794, 2796, 749, 750, 751, 752, 2802, 758, 759, 2806, 2809, 762, 2811, 764, 765, 2814, 2815, 2819, 775, 2824, 777, 778, 2825, 2827, 784, 2835, 2839, 792, 793, 2841, 797, 2846, 799, 2848, 802, 803, 804, 805, 2850, 2854, 2857, 812, 2860, 2862, 817, 818, 2866, 820, 2867, 822, 823, 2868, 2872, 2873, 827, 2874, 2876, 830, 2879, 832, 833, 2882, 837, 2887, 2888, 2891, 844, 2892, 2893, 847, 848, 849, 2898, 851, 2899, 2902, 2905, 858, 860, 2911, 2912, 870, 875, 2923, 2925, 878, 2929, 2931, 2932, 890, 891, 894, 895, 897, 898, 2947, 2950, 906, 2955, 908, 2956, 910, 2957, 912, 2959, 914, 2962, 916, 917, 2963, 2965, 2968, 921, 2970, 924, 925, 2972, 2974, 929, 930, 2977, 933, 935, 2983, 2986, 943, 948, 2997, 950, 2998, 956, 957, 964, 975, 976, 977, 981, 982, 983, 985, 986, 988, 990, 993, 996, 999, 1001, 1004, 1007, 1012, 1014, 1019, 1020, 1021, 1022, 1025, 1027, 1030, 1041, 1044, 1046, 1047, 1048, 1058, 1059, 1061, 1062, 1063, 1065, 1066, 1068, 1071, 1074, 1075, 1078, 1080, 1082, 1085, 1090, 1093, 1094, 1096, 1099, 1100, 1105, 1106, 1108, 1111, 1115, 1117, 1120, 1121, 1122, 1127, 1133, 1134, 1138, 1144, 1150, 1153, 1159, 1163, 1166, 1171, 1173, 1176, 1178, 1179, 1182, 1183, 1187, 1190, 1191, 1195, 1196, 1197, 1200, 1204, 1208, 1211, 1221, 1224, 1227, 1230, 1232, 1235, 1238, 1245, 1246, 1247, 1250, 1260, 1263, 1264, 1271, 1276, 1278, 1280, 1282, 1284, 1287, 1288, 1289, 1290, 1291, 1292, 1294, 1299, 1309, 1313, 1315, 1321, 1325, 1326, 1327, 1337, 1343, 1346, 1351, 1355, 1356, 1357, 1364, 1365, 1367, 1369, 1375, 1377, 1378, 1387, 1389, 1390, 1398, 1400, 1409, 1411, 1414, 1416, 1417, 1418, 1420, 1424, 1426, 1434, 1435, 1436, 1443, 1444, 1445, 1447, 1448, 1449, 1451, 1456, 1459, 1462, 1465, 1467, 1468, 1471, 1474, 1476, 1483, 1486, 1490, 1491, 1498, 1504, 1505, 1519, 1521, 1525, 1529, 1530, 1531, 1532, 1533, 1538, 1539, 1541, 1542, 1543, 1544, 1546, 1549, 1551, 1553, 1555, 1556, 1562, 1565, 1570, 1580, 1582, 1583, 1584, 1586, 1589, 1592, 1594, 1595, 1604, 1605, 1615, 1616, 1619, 1620, 1621, 1623, 1628, 1639, 1641, 1642, 1644, 1647, 1648, 1649, 1652, 1657, 1658, 1659, 1660, 1663, 1665, 1669, 1671, 1672, 1676, 1679, 1681, 1683, 1685, 1690, 1693, 1695, 1696, 1700, 1701, 1703, 1707, 1711, 1712, 1716, 1717, 1718, 1720, 1721, 1723, 1724, 1726, 1732, 1735, 1737, 1740, 1742, 1745, 1747, 1749, 1757, 1758, 1764, 1769, 1770, 1771, 1775, 1777, 1782, 1783, 1784, 1785, 1787, 1791, 1794, 1795, 1796, 1801, 1804, 1806, 1809, 1810, 1818, 1820, 1822, 1823, 1825, 1826, 1827, 1828, 1831, 1836, 1837, 1838, 1839, 1840, 1841, 1844, 1846, 1847, 1848, 1850, 1855, 1860, 1863, 1864, 1865, 1869, 1871, 1875, 1876, 1878, 1881, 1882, 1887, 1888, 1889, 1890, 1892, 1894, 1896, 1898, 1899, 1904, 1907, 1908, 1911, 1913, 1916, 1918, 1919, 1920, 1921, 1923, 1926, 1928, 1933, 1935, 1936, 1938, 1939, 1940, 1941, 1943, 1949, 1953, 1955, 1957, 1969, 1970, 1971, 1972, 1973, 1975, 1976, 1977, 1980, 1981, 1983, 1985, 1987, 1989, 1993, 2000, 2006, 2012, 2014, 2015, 2020, 2021, 2022, 2031, 2038, 2042, 2043, 2044, 2045]
print("A set of nouns with complete support under both role meanings:")
both_complete_nouns = [n for i, n in enumerate(nouns) if i in a and i in b]
print(both_complete_nouns)
print(len(both_complete_nouns))
print("#instances formed by both_complete_nouns: ", 
      len([t for t in tuples if t[0] in both_complete_nouns and t[1] in both_complete_nouns]))

A set of nouns with complete support under both role meanings:
['accident', 'accordion', 'activity', 'ad', 'adult', 'afternoon', 'air', 'airborne', 'airplane', 'alley', 'amount', 'angel', 'animal', 'anvil', 'apron', 'aquarium', 'area', 'arm', 'armchair', 'art', 'artwork', 'assignment', 'athlete', 'attempt', 'attention', 'atv', 'atvs', 'audience', 'autumn', 'awning', 'b', 'baby', 'back', 'background', 'backpack', 'bag', 'balance', 'balcony', 'bale', 'ball', 'balloon', 'banana', 'band', 'banister', 'banjo', 'banner', 'bar', 'barbecue', 'barrel', 'barricade', 'barrier', 'base', 'basketball', 'bass', 'bat', 'battle', 'beach', 'beam', 'beanbag', 'bear', 'beard', 'bed', 'beer', 'belly', 'belonging', 'belt', 'bench', 'bend', 'beverage', 'bicycle', 'bicycling', 'bicyclist', 'bike', 'bin', 'bird', 'blade', 'blanket', 'bleacher', 'block', 'blow', 'bmx', 'board', 'boardwalk', 'boat', 'body', 'bodyboard', 'bone', 'bongo', 'book', 'bookbag', 'boot', 'booth', 'bottle', 'bottom', 'bounce', 'bow', 'bo

In [45]:
heatmap = {}
for transpose in [False, True]:
    train_triplets = []
    for subj, obj, r in tuples:
        train_triplets.append((n2i[subj], n2i[obj], r2i[r]))
    if transpose: train_triplets = Transpose(train_triplets)

    c = Counter(train_triplets)
    print(c.most_common(3))

    prefix = "image" if transpose else "linguistic"
    heatmap[prefix] = np.zeros((len(both_complete_nouns), len(both_complete_nouns)))
    X = []
    for k, o1 in enumerate(both_complete_nouns):
        for l, o2 in enumerate(both_complete_nouns):
            x = c[(n2i[o1], n2i[o2], 0)] + c[(n2i[o1], n2i[o2], 1)]
            X.append(x)
            heatmap[prefix][k][l] = x


[((1553, 231, 0), 450), ((2963, 231, 0), 298), ((1553, 2338, 0), 276)]
[((1553, 231, 0), 450), ((2963, 231, 0), 298), ((1553, 2338, 0), 276)]


In [46]:
dead = []
iters = 0
killed_this_iter = True
while np.sum(heatmap['image']) * np.sum(heatmap['linguistic']) > 0 and killed_this_iter:
    killed_this_iter = False
    sum_axis0, sum_axis1 = np.sum(heatmap['image'], axis=0), np.sum(heatmap['image'], axis=1)
    for i, s in enumerate(sum_axis0):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop row i
                heatmap['image'][i][j] = 0
                # drop row i and col i
                heatmap['linguistic'][i][j] = 0
                heatmap['linguistic'][j][i] = 0

            dead.append(i)
            killed_this_iter = True
    for i, s in enumerate(sum_axis1):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop col i
                heatmap['image'][j][i] = 0
                # drop row i and col i
                heatmap['linguistic'][i][j] = 0
                heatmap['linguistic'][j][i] = 0
            dead.append(i)
            killed_this_iter = True

    sum_axis0, sum_axis1 = np.sum(heatmap['linguistic'], axis=0), np.sum(heatmap['linguistic'], axis=1)
    for i, s in enumerate(sum_axis0):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop row i
                heatmap['linguistic'][i][j] = 0
                # drop row i and col i
                heatmap['image'][i][j] = 0
                heatmap['image'][j][i] = 0

            dead.append(i)
            killed_this_iter = True
    for i, s in enumerate(sum_axis1):
        if i in dead: continue
        if s == 0:
            for j in range(len(both_complete_nouns)):
                # drop col i
                heatmap['linguistic'][j][i] = 0
                # drop row i and col i
                heatmap['image'][i][j] = 0
                heatmap['image'][j][i] = 0
            dead.append(i)
            killed_this_iter = True

    iters += 1
    print(f"dead nouns = {len(dead)}")
    print(f"""finish iter {iters}, 
          remaining examples = {np.sum(heatmap['image'])} (image), 
          {np.sum(heatmap['linguistic'])} (linguistic)""")    


dead nouns = 69
finish iter 1, 
          remaining examples = 30398.0 (image), 
          30398.0 (linguistic)
dead nouns = 74
finish iter 2, 
          remaining examples = 30218.0 (image), 
          30218.0 (linguistic)
dead nouns = 77
finish iter 3, 
          remaining examples = 30169.0 (image), 
          30169.0 (linguistic)
dead nouns = 78
finish iter 4, 
          remaining examples = 30162.0 (image), 
          30162.0 (linguistic)
dead nouns = 78
finish iter 5, 
          remaining examples = 30162.0 (image), 
          30162.0 (linguistic)


In [47]:
alive = set(list(range(len(both_complete_nouns)))) - set(dead)
alive = sorted(list(alive))
print("nouns with complete support".format(alive))
print(len(alive))
rows = np.array(alive, dtype=np.intp)
columns = np.array(alive, dtype=np.intp)

bijective_heatmap = heatmap['linguistic'][np.ix_(rows, columns)]
print("sanity check (linguistic): every row sum or col sum is positive")
print(np.min(np.sum(bijective_heatmap, axis=0)), np.min(np.sum(bijective_heatmap, axis=1)))

bijective_heatmap = heatmap['image'][np.ix_(rows, columns)]
print("sanity check (image): every row sum or col sum is positive")
print(np.min(np.sum(bijective_heatmap, axis=0)), np.min(np.sum(bijective_heatmap, axis=1)))

crux = alive # They are both_complete_nouns' indices
print(len(crux))
crux_nouns = [both_complete_nouns[i] for i in crux]


nouns with complete support
872
sanity check (linguistic): every row sum or col sum is positive
1.0 1.0
sanity check (image): every row sum or col sum is positive
1.0 1.0
872


In [48]:
for transpose in [False, True]:
    print("role intrinsic meanings: {} position\n".format("image" if transpose else "linguistic"))
    SUBJ, OBJ = [], []
    tuples, images, captions = [], [], []
    for unmapped_r in extracted_tuples:
        if not unmapped_r in positional_rel_v: continue
        
        r = map[unmapped_r]

        for a in extracted_tuples[unmapped_r]:
            subj, obj, _ = a[-1]
            #singularize
            singlular_subj = Word(subj).lemmatize()
            if not singlular_subj == False: subj = singlular_subj
            singlular_obj = Word(obj).lemmatize() 
            if not singlular_obj == False: obj = singlular_obj

            if subj in crux_nouns and obj in crux_nouns:
                SUBJ.append(subj)
                OBJ.append(obj)
                tuples.append((subj, obj, r))
                captions.append(a[1])
                images.append(a[0])
    
    nouns = sorted(list(set(SUBJ).union(set(OBJ))))
    num_nouns = len(nouns)
    print("#unique concepts = ", num_nouns)
    num_relations = len(relations)
    print("#instances = ", len(tuples))
    print("#unique instances = ", len(set(tuples)))
    print("#unique images = ", len(set(images)))
    print("#unique captions = ", len(set(captions)))

    train_triplets = [] # convert tuple elements to indices
    n2i = {n:i for i, n in enumerate(nouns)}
    r2i = {r:i for i, r in enumerate(relations)}
    print(len(r2i), len(n2i), "\n")
    for subj, obj, r in tuples:
        train_triplets.append((n2i[subj], n2i[obj], r2i[r]))
    if transpose: train_triplets = Transpose(train_triplets)

    # Sanity check crux_nouns are indeed complete under both linguistic & image positional roles
    print("#unique O1 = ", len(set([t[0] for t in train_triplets])))
    print("#unique O2 = ", len(set([t[1] for t in train_triplets])))
    df = pd.DataFrame(train_triplets, columns =['O1', 'O2', 'R'])
    for f in entropy_funcs:
        score = eval(f)
        print(f.split("(")[0], ": ", score, f"({round(score/np.log(2), 2)})")  
    print("--------------------------------------------------------\n\n")

role intrinsic meanings: linguistic position

#unique concepts =  872
#instances =  30162
#unique instances =  8797
#unique images =  16132
#unique captions =  30102
2 872 

#unique O1 =  872
#unique O2 =  872
concept_role_index_entropy :  0.21561153686435008 (0.31)
--------------------------------------------------------


role intrinsic meanings: image position

#unique concepts =  872
#instances =  30162
#unique instances =  8797
#unique images =  16132
#unique captions =  30102
2 872 

#unique O1 =  872
#unique O2 =  872
concept_role_index_entropy :  0.2629067575688466 (0.38)
--------------------------------------------------------




In [49]:
if transpose:
    print(len(train_triplets))
    print("train_triplets under {} positional roles:".format("image" if transpose else "linguistic"))
    print(train_triplets)
    print("unique train_triplets = {}".format(len(set(train_triplets))))
print("diagonal supports: ", set([nouns[t[0]] for t in train_triplets if t[0] == t[1]]))

30162
train_triplets under image positional roles:
[(868, 259, 0), (447, 427, 0), (447, 606, 0), (660, 718, 0), (432, 392, 0), (432, 392, 0), (432, 860, 0), (432, 392, 0), (432, 146, 0), (312, 619, 0), (312, 142, 0), (312, 142, 0), (93, 561, 0), (234, 611, 0), (234, 521, 0), (432, 611, 0), (432, 803, 0), (432, 611, 0), (864, 740, 0), (573, 116, 0), (432, 61, 0), (432, 61, 0), (432, 61, 0), (432, 61, 0), (3, 146, 0), (471, 370, 0), (529, 746, 0), (432, 99, 0), (432, 99, 0), (777, 432, 0), (777, 28, 0), (777, 28, 0), (154, 646, 0), (387, 646, 0), (387, 646, 0), (282, 191, 0), (529, 671, 0), (432, 290, 0), (154, 622, 0), (154, 476, 0), (432, 109, 0), (432, 807, 0), (234, 319, 0), (694, 326, 0), (67, 611, 0), (447, 740, 0), (432, 428, 0), (526, 414, 0), (864, 67, 0), (432, 64, 0), (432, 64, 0), (98, 671, 0), (234, 51, 0), (193, 61, 0), (224, 342, 0), (526, 439, 0), (526, 33, 0), (850, 432, 0), (755, 740, 0), (154, 28, 0), (154, 28, 0), (93, 740, 0), (154, 740, 0), (526, 740, 0), (432, 99, 

### Save Example Images

In [10]:
from diffusers.utils import make_image_grid
from datetime import datetime
import pytz
timezone = pytz.timezone('America/New_York') 
date = datetime.now(timezone).strftime("%m%d_%H%M%S")
example_dir = "../data/flickr30k/img_examples/"
images, texts = [], []
for e in random.sample(examples, 32):
    images.append(Image.open(os.path.join(
        "/data/yingshac/clevr_control/data/",
        e[1]
    )).convert("RGB").resize((32, 32)))
    texts.append(e[0])

image_grid = make_image_grid(images, rows=16, cols=math.ceil(len(images)/16))
image_grid.save("{}/{}.png".format(example_dir, date))
with open("{}/{}.txt".format(example_dir, date), "w") as f:
    f.write("\n".join(texts))

